<a href="https://colab.research.google.com/github/candiceevemiller/self-driving-cars/blob/main/speedestimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
path = 'gdrive/MyDrive/self-driving-cars/data/train/train.mp4'

vidObj = cv2.VideoCapture(path)

success = 1
video_frames = []
count = 0

while success:
    print(f'appending frame {count}')
    success, frame = vidObj.read()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        video_frames.append(frame[150:350]) # crop hood and sky
        count += 1

Streaming output truncated to the last 5000 lines.
appending frame 15401
appending frame 15402
appending frame 15403
appending frame 15404
appending frame 15405
appending frame 15406
appending frame 15407
appending frame 15408
appending frame 15409
appending frame 15410
appending frame 15411
appending frame 15412
appending frame 15413
appending frame 15414
appending frame 15415
appending frame 15416
appending frame 15417
appending frame 15418
appending frame 15419
appending frame 15420
appending frame 15421
appending frame 15422
appending frame 15423
appending frame 15424
appending frame 15425
appending frame 15426
appending frame 15427
appending frame 15428
appending frame 15429
appending frame 15430
appending frame 15431
appending frame 15432
appending frame 15433
appending frame 15434
appending frame 15435
appending frame 15436
appending frame 15437
appending frame 15438
appending frame 15439
appending frame 15440
appending frame 15441
appending frame 15442
appending frame 15443
app

In [3]:
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

In [4]:
video_frames = np.array(video_frames)

In [5]:
GAMMA_FACTOR = 1.75

video_frames = adjust_gamma(video_frames, GAMMA_FACTOR)

In [6]:
frame_pairs = []
for index, frame in enumerate(video_frames):
    if index > 1:
        frame_pairs.append(np.dstack((video_frames[index-2], video_frames[index-1], frame)))
        # frame_pairs.append([video_frames[index-1], frame])
frame_pairs = np.array(frame_pairs)

In [7]:
import matplotlib.pyplot as plt
plt.imshow(frame_pairs[0])

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: x/255),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', input_shape = (200,640,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1, activation='relu')
])

In [10]:
targets = np.fromfile('gdrive/MyDrive/self-driving-cars/data/train/train.txt', sep='\n')
targets

array([28.105569, 28.105569, 28.106527, ...,  2.292917,  2.2606  ,
        2.206759])

In [11]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [12]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=root_mean_squared_error)

In [13]:
history = model.fit(frame_pairs, targets, batch_size=32, epochs=10,
                    validation_split=.3,)

Epoch 1/10
447/447 [==============================] - 30s 50ms/step - loss: 16.7342 - root_mean_squared_error: 2.8694 - val_loss: 52.2408 - val_root_mean_squared_error: 6.4595
Epoch 2/10
447/447 [==============================] - 20s 45ms/step - loss: 4.7373 - root_mean_squared_error: 1.6252 - val_loss: 38.6075 - val_root_mean_squared_error: 5.4403
Epoch 3/10
447/447 [==============================] - 20s 45ms/step - loss: 3.6878 - root_mean_squared_error: 1.4278 - val_loss: 30.3974 - val_root_mean_squared_error: 4.7174
Epoch 4/10
447/447 [==============================] - 20s 45ms/step - loss: 3.4362 - root_mean_squared_error: 1.3776 - val_loss: 24.8477 - val_root_mean_squared_error: 4.1856
Epoch 5/10
447/447 [==============================] - 20s 45ms/step - loss: 2.9993 - root_mean_squared_error: 1.2880 - val_loss: 25.0966 - val_root_mean_squared_error: 4.2140
Epoch 6/10
447/447 [==============================] - 20s 45ms/step - loss: 2.6858 - root_mean_squared_error: 1.2150 - val_l